# Ensemble methods. Exercises


In this section we have only two exercise:

1. Find the best three classifier in the stacking method using the classifiers from scikit-learn package.

2. Build arcing arc-x4 method. 

In [27]:
%store -r data_set
%store -r labels
%store -r test_data_set
%store -r test_labels
%store -r unique_labels

## Exercise 1: Find the best three classifier in the stacking method

Please use the following classifiers:

* Linear regression,
* Nearest Neighbors,
* Linear SVM,
* Decision Tree,
* Naive Bayes,
* QDA.

In [28]:
import numpy as np
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [29]:
def build_classifiers():
    
    # fill this part
    classifiers = [
        ('linear_regression', LinearRegression()),
        ('neighbors', KNeighborsClassifier()),
        ('svm', SVC()),
        ('tree', DecisionTreeClassifier()),
        ('naive_bayes', GaussianNB()),
        ('qda', QuadraticDiscriminantAnalysis()),
    ]
    
    for clf in classifiers:
        clf[1].fit(data_set, labels)
        
    return classifiers

In [30]:
def build_stacked_classifier(classifiers):
    output = []
    for classifier in classifiers:
        output.append(classifier.predict(data_set))
    output = np.array(output).reshape((130,3))
    
    # stacked classifier part:
    stacked_classifier = DecisionTreeClassifier() # set here
    stacked_classifier.fit(output.reshape((130,3)), labels.reshape((130,)))
    test_set = []
    for classifier in classifiers:
        test_set.append(classifier.predict(test_data_set))
    test_set = np.array(test_set).reshape((len(test_set[0]),3))
    predicted = stacked_classifier.predict(test_set)
    return predicted

In [31]:
from itertools import permutations
from operator import itemgetter

In [32]:
classifiers = build_classifiers()
options = []
for prmt in permutations(classifiers, r=3):
    predicted = build_stacked_classifier([x[1] for x in prmt])
    accuracy = accuracy_score(test_labels, predicted)
    options.append((accuracy, ' > '.join([x[0] for x in prmt])))

options.sort(key=itemgetter(0), reverse=True)
for best in options[:10]:
    print(f'{best[0]:1.2f}: {best[1]}')

/home/gosia/miniconda3/envs/datascience/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.95: svm > tree > qda
0.95: tree > neighbors > qda
0.90: neighbors > tree > qda
0.90: neighbors > qda > tree
0.90: svm > neighbors > qda
0.90: tree > svm > qda
0.90: naive_bayes > neighbors > tree
0.90: naive_bayes > neighbors > qda
0.90: qda > neighbors > tree
0.85: neighbors > svm > tree


## Exercise 2: 

Use the boosting method and change the code to fullfilt the following requirements:

* the weights should be calculated as:
$w_{n}^{(t+1)}=\frac{1+ I(y_{n}\neq h_{t}(x_{n})}{\sum_{i=1}^{N}1+I(y_{n}\neq h_{t}(x_{n})}$,
* the prediction is done with a voting method.

In [33]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# prepare data set

def generate_data(sample_number, feature_number, label_number):
    data_set = np.random.random_sample((sample_number, feature_number))
    labels = np.random.choice(label_number, sample_number)
    return data_set, labels

labels = 2
dimension = 2
test_set_size = 1000
train_set_size = 5000
train_set, train_labels = generate_data(train_set_size, dimension, labels)
test_set, test_labels = generate_data(test_set_size, dimension, labels)

# init weights
number_of_iterations = 10
weights = np.ones((test_set_size,)) / test_set_size


def train_model(classifier, weights):
    return classifier.fit(X=test_set, y=test_labels, sample_weight=weights)

def calculate_error(model):
    predicted = model.predict(test_set)
    I=calculate_accuracy_vector(predicted, test_labels)
    Z=np.sum(I)
    return (1+Z)/1.0

def calculate_accuracy_vector(predicted, labels):
    result = []
    for i in range(len(predicted)):
        if predicted[i] == labels[i]:
            result.append(0)
        else:
            result.append(1)
    return result


Fill the two functions below:

In [34]:
def set_new_weights(model):
    # fill the code here (two lines)
    W = np.array(calculate_accuracy_vector(model.predict(test_set), test_labels))
    return (W + 1) / W.sum()

Train the classifier with the code below:

In [35]:
classifier = DecisionTreeClassifier(max_depth=1, random_state=1)
classifier.fit(X=train_set, y=train_labels)
alphas = []
classifiers = []
for iteration in range(number_of_iterations):
    model = train_model(classifier, weights)
    weights = set_new_weights(model)
    classifiers.append(model)

print(weights)


validate_x, validate_label = generate_data(1, dimension, labels)

[0.002 0.002 0.004 0.002 0.002 0.004 0.002 0.002 0.004 0.002 0.004 0.002
 0.004 0.004 0.002 0.004 0.004 0.004 0.004 0.004 0.004 0.004 0.004 0.002
 0.002 0.002 0.004 0.002 0.002 0.004 0.002 0.004 0.004 0.002 0.002 0.002
 0.002 0.004 0.004 0.002 0.002 0.004 0.002 0.004 0.004 0.004 0.004 0.004
 0.004 0.004 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.004 0.004 0.002
 0.004 0.002 0.004 0.004 0.004 0.002 0.002 0.002 0.004 0.004 0.004 0.002
 0.002 0.002 0.004 0.004 0.002 0.002 0.004 0.004 0.004 0.004 0.002 0.002
 0.004 0.002 0.004 0.004 0.002 0.004 0.004 0.004 0.004 0.004 0.002 0.004
 0.004 0.002 0.002 0.002 0.004 0.002 0.002 0.002 0.004 0.002 0.002 0.004
 0.004 0.004 0.002 0.002 0.004 0.002 0.002 0.002 0.004 0.004 0.002 0.002
 0.002 0.004 0.002 0.002 0.004 0.004 0.004 0.004 0.002 0.002 0.002 0.002
 0.004 0.002 0.002 0.002 0.004 0.002 0.002 0.002 0.004 0.004 0.002 0.002
 0.004 0.004 0.004 0.004 0.004 0.002 0.004 0.004 0.002 0.002 0.002 0.004
 0.004 0.002 0.004 0.004 0.002 0.004 0.004 0.004 0.

Set the validation data set:

In [36]:
validate_x, validate_label = generate_data(1, dimension, labels)

Fill the prediction code:

In [37]:
def get_prediction(x):
    # fill the code here (5-6 lines)
    return classifiers[-1].predict(x)

Test it:

In [38]:
prediction = get_prediction(validate_x)[0]

print('input', validate_x)
print('output:', prediction)
print('proper output:', validate_label)


input [[0.61474167 0.83843963]]
output: 1
proper output: [1]
